# Correlating Returns

In [75]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Load API Keys from Environment Variables

In [84]:
# Load .env enviroment variables
load_dotenv("example.env")

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

## Get AAPL Returns for Past Month

In [85]:
# Set the ticker
ticker = "AAPL"

# Set timeframe to '1D'
timeframe = "1D"

# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp("2020-08-25", tz="America/New_York").isoformat()
past_date = pd.Timestamp("2020-07-25", tz="America/New_York").isoformat()

# Get 4 weeks worth of historical data for AAPL
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=past_date,
    end=current_date,
    after=None,
    until=None,
).df

# Display data
df.head()

AAPL                                     
                              open      high     low    close    volume
2020-07-27 00:00:00-04:00  374.840  379.6200  373.92  379.215  26920001
2020-07-28 00:00:00-04:00  377.470  378.1986  372.99  373.040  23023333
2020-07-29 00:00:00-04:00  375.000  380.9200  374.85  380.270  20387893
2020-07-30 00:00:00-04:00  376.750  385.1900  375.07  384.880  29577510
2020-07-31 00:00:00-04:00  411.535  425.6600  403.30  425.190  86940020

In [86]:
# Drop Outer Table Level
df = df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
df = df.drop(columns=["open", "high", "low", "volume"])

# Since this is daily data, we can keep only the date (remove the time) component of the data
df.index = df.index.date

# Display sample data
df.head()

,close
2020-07-27,379.215
2020-07-28,373.040
2020-07-29,380.270
2020-07-30,384.880
2020-07-31,425.190


In [87]:
# Use the `pct_change` function to calculate daily returns of AAPL
aapl_returns = df.pct_change().dropna()

# Display sample data
aapl_returns.head()

,close
2020-07-28,-0.016284
2020-07-29,0.019381
2020-07-30,0.012123
2020-07-31,0.104734
2020-08-03,0.024624


In [88]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=datetime.strftime(date, '%Y-%m-%dT00:00:00'),
            to=datetime.strftime(date, '%Y-%m-%dT00:00:00'),
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates

In [89]:
# Get first topic
aapl_headlines, dates = get_headlines("apple")

Fetching news about 'apple'
******************************
retrieving news from: 2020-08-25 00:00:00


NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 500 requests over a 24 hour period (250 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [82]:
# Get second topic
trade_headlines, _ = get_headlines("trade")

Fetching news about 'trade'
******************************
retrieving news from: 2020-08-25 00:00:00
retrieving news from: 2020-08-24 00:00:00
retrieving news from: 2020-08-23 00:00:00
retrieving news from: 2020-08-22 00:00:00
retrieving news from: 2020-08-21 00:00:00
retrieving news from: 2020-08-20 00:00:00
retrieving news from: 2020-08-19 00:00:00
retrieving news from: 2020-08-18 00:00:00
retrieving news from: 2020-08-17 00:00:00
retrieving news from: 2020-08-16 00:00:00
retrieving news from: 2020-08-15 00:00:00
retrieving news from: 2020-08-14 00:00:00
retrieving news from: 2020-08-13 00:00:00
retrieving news from: 2020-08-12 00:00:00
retrieving news from: 2020-08-11 00:00:00
retrieving news from: 2020-08-10 00:00:00
retrieving news from: 2020-08-09 00:00:00
retrieving news from: 2020-08-08 00:00:00
retrieving news from: 2020-08-07 00:00:00
retrieving news from: 2020-08-06 00:00:00
retrieving news from: 2020-08-05 00:00:00
retrieving news from: 2020-08-04 00:00:00
retrieving news f

In [69]:
# Get third topic
economy_headlines, _ = get_headlines("economy")

Fetching news about 'economy'
******************************
retrieving news from: 2020-08-25 00:00:00
retrieving news from: 2020-08-24 00:00:00
retrieving news from: 2020-08-23 00:00:00
retrieving news from: 2020-08-22 00:00:00
retrieving news from: 2020-08-21 00:00:00
retrieving news from: 2020-08-20 00:00:00
retrieving news from: 2020-08-19 00:00:00
retrieving news from: 2020-08-18 00:00:00
retrieving news from: 2020-08-17 00:00:00
retrieving news from: 2020-08-16 00:00:00
retrieving news from: 2020-08-15 00:00:00
retrieving news from: 2020-08-14 00:00:00
retrieving news from: 2020-08-13 00:00:00
retrieving news from: 2020-08-12 00:00:00
retrieving news from: 2020-08-11 00:00:00
retrieving news from: 2020-08-10 00:00:00
retrieving news from: 2020-08-09 00:00:00
retrieving news from: 2020-08-08 00:00:00
retrieving news from: 2020-08-07 00:00:00
retrieving news from: 2020-08-06 00:00:00
retrieving news from: 2020-08-05 00:00:00
retrieving news from: 2020-08-04 00:00:00
retrieving news

In [91]:
# Get fourth topic
# iphone_headlines, _ = get_headlines("iphone")
iphone_headlines

[['How This Safari Bug Can Expose Files on Your Mac or iPhone',
  'iPhone 12 launch date leaks as iPhone XR, 11 Pro rumoured to be axed - What Hi-Fi?',
  'Nomad’s Base Station Pro with Aira FreePower tech finally realizes the promise of wireless charging',
  'Alleged Screenshots From iPhone 12 Pro Max Settings Suggest 120Hz Display, LiDAR Functionality',
  "Samsung's Galaxy Buds+ drop to $130 on Amazon",
  'TSMC Details Potential iPhone 12 A14 Performance and Upcoming 3nm Process',
  'Apple to open India online store in September: Bloomberg News - Reuters',
  'Netgear debuts a new 15.6-inch Meural WiFi Photo Frame with automatic album syncing',
  'Can Thieves Crack 6-Digit iPhone Passcodes?',
  'DigiTimes: Third Quarter MacBook Sales Up 20% Compared to Last Year',
  'iPhone: How to block texts',
  'Apple might finally open its online store in India next month',
  "Apple Pay Support Goes Live for Germany's Sparkasse Debit Girocard",
  "Judge 'inclined' to block Apple's move to hamper Ep

In [ ]:
# Get fifth topic
gold_headlines, _ = get_headlines("gold")

In [ ]:
# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [36]:
# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [92]:
# Get averages of each topics sentiment
aapl_avg = headline_sentiment_summarizer_avg(aapl_headlines)
trade_avg = headline_sentiment_summarizer_avg(trade_headlines)
economy_avg = headline_sentiment_summarizer_avg(economy_headlines)
iphone_avg = headline_sentiment_summarizer_avg(iphone_headlines)
gold_avg = headline_sentiment_summarizer_avg(gold_headlines)

In [93]:
# Combine Sentiment Averages into DataFrame
topic_sentiments = pd.DataFrame(
    {
        "aapl_avg": aapl_avg,
        "trade_avg": trade_avg,
        "economy_avg": economy_avg,
        "iphone_avg": iphone_avg,
        "gold_avg": gold_avg,
    }
)

In [95]:
# Set the index value of the sentiment averages DataFrame to be the series of dates.
topic_sentiments.index = pd.to_datetime(dates)

In [96]:
# Merge with AAPL returns
topic_sentiments = aapl_returns.join(topic_sentiments).dropna(how="any")

# Display data
display(topic_sentiments)

,close,aapl_avg,trade_avg,economy_avg,iphone_avg,gold_avg
2020-07-28,-0.016284,0.063105,0.060520,0.096730,0.143260,0.109165
2020-07-29,0.019381,-0.047380,-0.018815,0.030850,0.074155,0.142205
2020-07-30,0.012123,0.104800,-0.088845,-0.094395,0.148510,0.005910
2020-07-31,0.104734,0.206250,0.063255,-0.088030,0.319570,0.209600
2020-08-03,0.024624,-0.037105,0.100315,-0.012805,-0.006315,0.021625
2020-08-04,0.007230,0.042490,-0.129925,0.038625,0.179645,0.068175
2020-08-05,0.003484,0.042465,-0.068375,-0.046125,-0.042000,0.112405
2020-08-06,0.035293,0.028655,-0.012540,-0.080740,0.123580,0.075495
2020-08-07,-0.025160,0.049720,0.195485,0.056365,-0.047100,0.145940
2020-08-10,0.014694,-0.108200,0.049335,-0.040755,-0.013095,0.113015


In [97]:
# Correlate the headlines' sentiment to returns
topic_sentiments.corr().style.background_gradient()

,close,aapl_avg,trade_avg,economy_avg,iphone_avg,gold_avg
close,1.000000,0.194562,0.002357,-0.362245,0.534055,-0.141538
aapl_avg,0.194562,1.000000,-0.290992,-0.056692,0.459510,0.036016
trade_avg,0.002357,-0.290992,1.000000,0.233496,-0.420102,0.171642
economy_avg,-0.362245,-0.056692,0.233496,1.000000,-0.136589,0.388511
iphone_avg,0.534055,0.459510,-0.420102,-0.136589,1.000000,0.021426
gold_avg,-0.141538,0.036016,0.171642,0.388511,0.021426,1.000000
